# **Modelo 2 (Generacion de Poemas) GPT2**


---

In [ ]:
pip install transformers -U

In [2]:
# Basicas
import pandas as pd
import numpy as np
import os
import time
import datetime
# Pytorch
import torch
from torch import nn
from torch.utils.data import Dataset, TensorDataset, DataLoader
import torch.nn.functional as F
from torch.utils.data import Dataset, random_split, DataLoader, RandomSampler, SequentialSampler
# Texto
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import random
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
from transformers import pipeline

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
AVAIL_GPUS = min(1, torch.cuda.device_count())
def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

# **Datos**

In [3]:
url = 'https://raw.githubusercontent.com/andreamorgar/poesIA/master/data/poems.csv'
poems_df = pd.read_csv(url)
poems_df = poems_df.dropna()

In [4]:
# Filtrar poemas grandes
poems_df['string'] = poems_df.apply(lambda row: f'\n{row["title"]}\n{row["content"]}', axis=1)
poems_df['length'] = poems_df.string.map(len)
MAX_POEM_LENGTH=1000
poems_filtered = poems_df[poems_df.length<MAX_POEM_LENGTH]
#_ , poems_filtered = train_test_split(poems_filtered, test_size = 0.5 ,shuffle=True)
poems_filtered

,author,content,title,string,length
1,Marilina Rébora,"\n\nPorque si tú no velas, vendré como ladrón;...",PORQUE SI TÚ NO VELAS,\nPORQUE SI TÚ NO VELAS\n\n\nPorque si tú no v...,714
2,Antonio Colinas,"\n\nPequeña de mis sueños, por tu piel las pal...",POEMA DE LA BELLEZA CAUTIVA QUE PERDÍ,\nPOEMA DE LA BELLEZA CAUTIVA QUE PERDÍ\n\n\nP...,644
3,José María Hinojosa,\n\nLos dedos de la nieve\nrepiquetearon\nen e...,SENCILLEZ,\nSENCILLEZ\n\n\nLos dedos de la nieve\nrepiqu...,251
4,Rubén Izaguirre Fiallos,"Naciste en Armenia,\npero te fuiste a vivir al...",Breve Carta a Consuelo Suncín,\nBreve Carta a Consuelo Suncín\nNaciste en Ar...,390
5,Leopoldo María Panero,\n\nOscuridad nieve buitres desespero oscurida...,PASADIZO SECRETO,\nPASADIZO SECRETO\n\n\nOscuridad nieve buitre...,325
...,...,...,...,...,...
5127,Ángel González,\n\nCruzas por el crepúsculo.\nEl aire\ntienes...,BOSQUE,\nBOSQUE\n\n\nCruzas por el crepúsculo.\nEl ai...,425
5129,David Escobar Galindo,\n\nNada es memoria: todo es invención.\nLo qu...,Nada es memoria,\nNada es memoria\n\n\nNada es memoria: todo e...,234
5130,amistad,\nFelicidad: Muy dentro de tí.\nSerenidad: En ...,Esto es todo lo que deseo para tí,\nEsto es todo lo que deseo para tí\n\nFelicid...,494
5131,Octavio Paz,\nMis manos \nabren las cortinas de tu ser \nt...,Palpar,\nPalpar\n\nMis manos \nabren las cortinas de ...,159


In [5]:
print(list(poems_filtered['string'])[20])


PALMA SOLA


La palma que está en el patio
nació sola;
creció sin que yo la viera,
creció sola;
bajo la luna y el sol,
vive sola.

Con su largo cuerpo fijo,
palma sola;
sola en el patio sellado,
siempre sola,
guardián del atardecer,
sueña sola.

La palma sola soñando,
palma sola,
que va libre por el viento,
libre y sola,
suelta de raíz y tierra,
suelta y sola,
cazadora de las nubes,
palma sola,
palma sola,
palma.


# **Tokens para los datos (modelo DeepESP/gpt2-spanish)**

In [6]:
df = poems_filtered

In [7]:
tokenizer = GPT2Tokenizer.from_pretrained("DeepESP/gpt2-spanish")
special_tokens_dict = {
    'bos_token': '<BOS>', 
    'eos_token': '<EOS>', 
    'pad_token': '<PAD>'}
num_added_tokens = tokenizer.add_special_tokens(special_tokens_dict)

In [8]:
class DataTokens(Dataset):  
    def __init__(self, data, tokenizer, gpt2_type="gpt2", max_length=1000):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.attn_masks = []
        for row in df['string']:
          encodings_dict = self.tokenizer('<BOS>' + row + '<EOS>',
                                     truncation=True,
                                     max_length=max_length,
                                     padding='max_length')
          self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
          self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

dataset = DataTokens(df['string'], tokenizer, gpt2_type="DeepESP/gpt2-spanish")

In [9]:
# Train / Validation Split
def train_val_split(split, dataset):
    train_size = int(split * len(dataset))
    val_size = len(dataset) - train_size
    return train_size, val_size

In [10]:
RANDOM_SEED = 73
torch.cuda.manual_seed_all(RANDOM_SEED)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [11]:
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

# **Reentrenamiento para el modelo (DeepESP/gpt2-spanish)**

In [12]:
configuration = GPT2Config(vocab_size=len(tokenizer), n_positions=1000).from_pretrained("DeepESP/gpt2-spanish", output_hidden_states=True)
poem_stanza_model = GPT2LMHeadModel.from_pretrained("DeepESP/gpt2-spanish", config=configuration)
poem_stanza_model.resize_token_embeddings(len(tokenizer))

Embedding(50260, 768)

In [13]:
def train(dataset, model, batch_size=32, epochs=5, learning_rate = 1e-4, eps = 1e-8, warmup_steps=50):
    # DataLoaders
    poem_line_train_size, poem_line_val_size = train_val_split(0.8, dataset)
    poem_line_train_dataset, poem_line_val_dataset = random_split(dataset, [poem_line_train_size, poem_line_val_size])
    poem_stanza_train_dataloader = DataLoader(poem_line_train_dataset,
                              sampler=RandomSampler(poem_line_train_dataset),
                              batch_size=batch_size)
    poem_stanza_val_dataloader = DataLoader(poem_line_val_dataset,
                                sampler=SequentialSampler(poem_line_val_dataset),
                                batch_size=batch_size)
    # Modelo
    model = model.cuda()
    optimizer = AdamW(model.parameters(), lr=learning_rate, eps=eps)
    total_steps = len(poem_stanza_train_dataloader) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps=warmup_steps,num_training_steps=total_steps)
    start_time = time.time()
    torch.cuda.empty_cache()
    # Entrenamiento
    for epoch_i in range(0, epochs):
      print(f'Epoch {epoch_i + 1} de {epochs}')
      t0 = time.time()
      total_train_loss = 0
      model.train()
      for step, batch in enumerate(poem_stanza_train_dataloader):
        b_input_ids = batch[0].to(device)
        b_masks = batch[1].to(device)
        model.zero_grad()
        outputs = model(b_input_ids,labels=b_input_ids,attention_mask=b_masks,token_type_ids=None)
        loss = outputs[0]
        batch_loss = loss.item()
        total_train_loss += batch_loss
        loss.backward()
        optimizer.step()
        scheduler.step()
      avg_train_loss = total_train_loss / len(poem_stanza_train_dataloader)
      training_time = format_time(time.time() - t0)
      print(f'Average Training Loss: {avg_train_loss}. Epoch Training Time: {training_time}')
      # Validacion
      t0 = time.time()
      model.eval()
      total_eval_loss = 0
      nb_eval_steps = 0
      for batch in poem_stanza_val_dataloader:
        torch.cuda.empty_cache()
        b_input_ids = batch[0].to(device)
        b_masks = batch[1].to(device)
        with torch.no_grad():
          outputs  = poem_stanza_model(b_input_ids,attention_mask=b_masks,labels=b_input_ids)
          loss = outputs[0]
        batch_loss = loss.item()
        total_eval_loss += batch_loss
      avg_val_loss = total_eval_loss / len(poem_stanza_val_dataloader)
      print(f'Average Validation Loss: {avg_val_loss}')
    print(f'Total Training Time: {format_time(time.time()-start_time)}')
    return model

In [ ]:
model = train(dataset, poem_stanza_model, epochs=10)
torch.save(model, 'modelo_gpt2_poesia.pt')

# **Generación de Poesía**

In [ ]:
model = torch.load('modelo_gpt2_poesia.pt')

In [ ]:
def generate(model,tokenizer,prompt,length=60,top_p=0.8,temperature=1.):
    #prompt = trad_es_en(prompt)[0]['translation_text']
    model.eval()
    generated_num = 0
    generated_list = []
    filter_value = -float("Inf")
    with torch.no_grad():
      entry_finished = False
      generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
      for i in range(length):
          outputs = model(generated, labels=generated)
          loss, logits = outputs[:2]
          logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)
          sorted_logits, sorted_indices = torch.sort(logits, descending=True)
          cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)
          sorted_indices_to_remove = cumulative_probs > top_p
          sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
          sorted_indices_to_remove[..., 0] = 0
          indices_to_remove = sorted_indices[sorted_indices_to_remove]
          logits[:, indices_to_remove] = filter_value
          next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
          generated = torch.cat((generated, next_token), dim=1)
          if next_token in tokenizer.encode("<|endoftext|>"): break 
      output_list = list(generated.squeeze().numpy())
      output_text = tokenizer.decode(output_list,skip_special_tokens=True)
      generated_list.append(output_text)
    #generated_list = trad_en_es(generated_list)[0]['translation_text']  
    return generated_list[0]

In [ ]:
Palabra = 'CIELO ESTRELLADO'
text = generate(model.to('cpu'), tokenizer,Palabra,temperature=0.7,length = 100,top_p = 0.8)
print(text)

In [ ]:
def Generate2(model,Palabra):
  input_ids = tokenizer.encode(Palabra, return_tensors="pt")
  output = model.generate(
      input_ids,
      num_beams=5,
      max_length=40,
      early_stopping=True,
      no_repeat_ngram_size=2,
      temperature=1.5
  )
  output = tokenizer.decode(output[0], skip_special_tokens=True)
  return output

In [ ]:
Palabra = 'CIELO ESTRELLADO'
text = Generate2(model.to('cpu'),Palabra)
print(text)